# Modèle pour la détermination du type de véhicule

### 1. Préparation des données

Importation des datasets

In [1]:
import os

#x_train = #TODO
y_train = [category.split('_')[0] for category in os.listdir('./result')]



print(y_train[:10])

['cars', 'cars', 'cars', 'cars', 'cars', 'cars', 'cars', 'cars', 'cars', 'cars']


Transformation : Niveau de gris sur tout le dataset

In [3]:
import cv2, os

output_folder = './gray_images'


for image_name in os.listdir('./result') :
    img = cv2.imread(f'./result/{image_name}')
    grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, grayFrame)


Transformation : ajout de bruit gaussien sur les images pour simuler une caméra de mauvaise qualité

In [4]:
# import cv2
# import os
import numpy as np
from PIL import Image
import random

def add_gaussian_noise(img, mean=0, std=25):
    """
    Ajoute un bruit gaussien à une image.
    """
    # Convertir l'image en tableau numpy
    img_array = np.array(img, dtype=np.float32)

    # Générer du bruit gaussien
    std = 10
    noise = np.random.normal(mean, std, img_array.shape)
    noisy_img = img_array + noise

    # Limiter les valeurs entre 0 et 255
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img

def process_images_with_effects(source_dir, target_dir, apply_blur=True, apply_noise=True):
    """
    Applique flou et bruit aux images redimensionnées.
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            file_path = os.path.join(source_dir, filename)
            save_path = os.path.join(target_dir, filename)

            # Charger l'image
            with Image.open(file_path) as img:
                # Convertir en BGR pour OpenCV
                img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

                # Appliquer bruit gaussien
                if apply_noise and random.random() > 0.3:  # 50% de chances d'ajouter du bruit
                    img_cv = add_gaussian_noise(img_cv, mean=0, std=random.randint(15, 40))

                # Convertir en RGB et sauvegarder
                final_img = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
                final_img.save(save_path)
                print(f"Processed and saved: {save_path}")

# Répertoires
source_directory = "./gray_images"  # Changez avec votre chemin
target_directory = "./noise_images"  # Répertoire cible

# Appliquer flou et bruit
process_images_with_effects(source_directory, target_directory, apply_blur=True, apply_noise=True)

Processed and saved: ./noise_images\cars_100_1.jpg
Processed and saved: ./noise_images\cars_100_2.jpg
Processed and saved: ./noise_images\cars_101_1.jpg
Processed and saved: ./noise_images\cars_102_1.jpg
Processed and saved: ./noise_images\cars_103_1.jpg
Processed and saved: ./noise_images\cars_103_2.jpg
Processed and saved: ./noise_images\cars_104_1.jpg
Processed and saved: ./noise_images\cars_104_2.jpg
Processed and saved: ./noise_images\cars_105_1.jpg
Processed and saved: ./noise_images\cars_105_2.jpg
Processed and saved: ./noise_images\cars_106_1.jpg
Processed and saved: ./noise_images\cars_107_1.jpg
Processed and saved: ./noise_images\cars_109_1.jpg
Processed and saved: ./noise_images\cars_109_2.jpg
Processed and saved: ./noise_images\cars_10_1.jpg
Processed and saved: ./noise_images\cars_110_1.jpg
Processed and saved: ./noise_images\cars_111_1.jpg
Processed and saved: ./noise_images\cars_112_1.jpg
Processed and saved: ./noise_images\cars_113_1.jpg
Processed and saved: ./noise_ima

Transformation : Redimensionnement pour que les images soient environ de la taille de la médiane des tailles des exemples

In [5]:
from PIL import Image
import os
import numpy as np
import random

def calculate_median_size(image_paths):
    """Calcule la médiane des tailles (largeur, hauteur) des images."""
    dimensions = []
    for path in image_paths:
        with Image.open(path) as img:
            dimensions.append(img.size)  # (width, height)
    dimensions = np.array(dimensions)
    median_width = int(np.median(dimensions[:, 0]))
    median_height = int(np.median(dimensions[:, 1]))
    return median_width, median_height

def add_random_variation(size, variation_percent=0.2):
    """
    Ajoute une variation aléatoire à une taille donnée.
    """
    factor = 1 + random.uniform(-variation_percent, variation_percent)
    return int(size * factor)

def resize_image_with_aspect_ratio(img, target_width, target_height):
    """
    Redimensionne une image en conservant le ratio d'aspect, en s'adaptant à la taille cible.
    """
    original_width, original_height = img.size
    aspect_ratio = original_width / original_height

    # Ajuster la largeur et la hauteur pour conserver le ratio
    if target_width / target_height > aspect_ratio:
        # Ajuste par la hauteur
        new_height = target_height
        new_width = int(new_height * aspect_ratio)
    else:
        # Ajuste par la largeur
        new_width = target_width
        new_height = int(new_width / aspect_ratio)

    return img.resize((new_width, new_height), Image.ANTIALIAS)

def resize_images(source_dir, target_dir, median_size, variation_percent=0.2):
    """
    Redimensionne les images en conservant leur ratio d'aspect, avec des tailles légèrement différentes.
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            file_path = os.path.join(source_dir, filename)
            with Image.open(file_path) as img:
                # Ajouter une variation aléatoire aux dimensions médianes
                target_width = add_random_variation(median_size[0], variation_percent)
                target_height = add_random_variation(median_size[1], variation_percent)

                # Redimensionner en conservant le ratio d'aspect
                resized_img = resize_image_with_aspect_ratio(img, target_width, target_height)

                # Sauvegarder l'image redimensionnée
                save_path = os.path.join(target_dir, filename)
                resized_img.save(save_path)
                print(f"Resized and saved: {save_path} -> {resized_img.size}")

# Répertoires
source_directory = "./noise_images"  # Changez avec votre chemin
target_directory = "./resized_images"  # Répertoire cible

dimension_images = "./AlgorithmePreProcess/dimension_images"

# Images avec tailles définies
defined_images = [os.path.join(dimension_images, f) for f in os.listdir(dimension_images)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

# Calcul de la taille médiane
median_size = calculate_median_size(defined_images)

# Redimensionnement des images avec une variation aléatoire de ±20 %
resize_images(source_directory, target_directory, median_size, variation_percent=0.2)


C:\Users\rapha\AppData\Local\Temp\ipykernel_27088\2944075056.py:41: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  return img.resize((new_width, new_height), Image.ANTIALIAS)


Resized and saved: ./resized_images\cars_100_1.jpg -> (35, 48)
Resized and saved: ./resized_images\cars_100_2.jpg -> (37, 49)
Resized and saved: ./resized_images\cars_101_1.jpg -> (38, 46)
Resized and saved: ./resized_images\cars_102_1.jpg -> (42, 30)
Resized and saved: ./resized_images\cars_103_1.jpg -> (42, 45)
Resized and saved: ./resized_images\cars_103_2.jpg -> (37, 47)
Resized and saved: ./resized_images\cars_104_1.jpg -> (48, 43)
Resized and saved: ./resized_images\cars_104_2.jpg -> (34, 31)
Resized and saved: ./resized_images\cars_105_1.jpg -> (32, 60)
Resized and saved: ./resized_images\cars_105_2.jpg -> (49, 45)
Resized and saved: ./resized_images\cars_106_1.jpg -> (34, 51)
Resized and saved: ./resized_images\cars_107_1.jpg -> (46, 33)
Resized and saved: ./resized_images\cars_109_1.jpg -> (20, 49)
Resized and saved: ./resized_images\cars_109_2.jpg -> (18, 48)
Resized and saved: ./resized_images\cars_10_1.jpg -> (39, 42)
Resized and saved: ./resized_images\cars_110_1.jpg -> (4

Transformation : UpSampling en retournant les images aléatoirement
Tous les exemples uniques de rotation/flip d'image :

| Rotation (k) | Flip Horizontal | Flip Vertical |
|--------------|-----------------|---------------|
| 0            | False           | False         |
| 0            | True            | False         |
| 0            | False           | True          |
| 1            | False           | False         |
| 1            | True            | False         |
| 2            | False           | False         |
| 2            | True            | False         |
| 3            | False           | False         |
| 3            | True            | False         |

In [27]:
import tensorflow as tf
import numpy as np

# def random_rotate_image(opencv_img):
#     img = tf.convert_to_tensor(opencv_img, dtype=tf.float32)
#     k = random.randint(1, 3)  # 1 = 90°, 2 = 180°, 3 = 270°
#     img = tf.image.rot90(img, k=k)
#     if k != 4:
#         img = tf.image.random_flip_left_right(img)
#         img = tf.image.random_flip_up_down(img)

#     img_rotated = img.numpy().astype(np.uint8)
#     return img_rotated
unique_combinations = [
    # (0, False, False),
    (0, True, False),
    (0, False, True),
    (1, False, False),
    (1, True, False),
    (2, False, False),
    (2, True, False),
    (3, False, False),
    (3, True, False)
]

def apply_transformations(img, rotation_count, flip_horizontal, flip_vertical):
    """
    Applique une transformation à l'image selon le nombre de rotations et les flips.
    """
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    img = tf.image.rot90(img, k=rotation_count)

    # Appliquer les flips
    if flip_horizontal:
        img = tf.image.flip_left_right(img)
    if flip_vertical:
        img = tf.image.flip_up_down(img)

    img_rotated = img.numpy().astype(np.uint8)
    return img_rotated

output_folder = './rotated_gray_images'
os.makedirs(output_folder, exist_ok=True)

for image_name in os.listdir('./resized_images') :
    img = cv2.imread(f'./resized_images/{image_name}')

    # On choisit deux transformations aléatoires
    random_choice1 = random.choice(unique_combinations)
    random_choice2 = random.choice(unique_combinations)

    # On vérifie que les choix sont différents
    while random_choice1 == random_choice2:
        random_choice2 = random.choice(unique_combinations)

    rotation_count1, flip_horizontal1, flip_vertical1 = random_choice1
    rotation_count2, flip_horizontal2, flip_vertical2 = random_choice2

    rotations_list = [apply_transformations(img, rotation_count1, flip_horizontal1, flip_vertical1), 
                      apply_transformations(img, rotation_count2, flip_horizontal2, flip_vertical2)]
    
    for i, rotated_image in enumerate(rotations_list) :
        output_image_name = f"{os.path.splitext(image_name)[0]}_{i}.png"
        output_path = os.path.join(output_folder, output_image_name)
        cv2.imwrite(output_path, rotated_image)

In [ ]:
# img = cv2.imread(f'./resized_images/cars_2_1.jpg')
# img = tf.convert_to_tensor(img, dtype=tf.float32)
# img = tf.image.rot90(img, k=2)
# img = tf.image.random_flip_up_down(img)
# img_rotated = img.numpy().astype(np.uint8)
# output_folder = './TEST_ROTATION'
# os.makedirs(output_folder, exist_ok=True)

# output_image_name = f"car_2_rotated_flip.png"
# output_path = os.path.join(output_folder, output_image_name)
# cv2.imwrite(output_path, img_rotated)

True

Création du modèle